In [1]:
function f(x_prev, v, dt)
    x_prev + v*dt
end

@code_native debuginfo=:none dump_module=false f(1.0, 2.0, 0.1)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vmulsd	xmm1, xmm1, xmm2
	vaddsd	xmm0, xmm1, xmm0
	pop	rbp
	ret
	nop	dword ptr [rax]


In [2]:
@fastmath function f_fastmath(x_prev, v, dt)
    x_prev + v*dt
end

@code_native debuginfo=:none dump_module=false f_fastmath(1.0, 2.0, 0.1)

	.text
	push	rbp
	mov	rbp, rsp
	mov	rax, qword ptr [r13 + 16]
	mov	rax, qword ptr [rax + 16]
	mov	rax, qword ptr [rax]
	vfmadd231sd	xmm0, xmm2, xmm1        # xmm0 = (xmm2 * xmm1) + xmm0
	pop	rbp
	ret
	nop	word ptr cs:[rax + rax]


In [3]:
function repeatf(f, n=10^6, x0=0.0, v=1.0, dt=1/n)
    x = x0
    for _ in 1:n
        x = f(x, v, dt)
    end
    x
end

@show repeatf(f)
@show repeatf(f_fastmath);

repeatf(f) = 1.000000000007918
repeatf(f_fastmath) = 1.0000000000006695


In [4]:
using BenchmarkTools

@btime repeatf(f)
@btime repeatf(f_fastmath);

  856.500 μs (0 allocations: 0 bytes)
  10.900 μs (0 allocations: 0 bytes)
